## Run MMseqs2 on the single mutation benchmark

In [1]:
from __future__ import annotations
import sys
import csv
import pandas as pd
from pathlib import Path

root_dir = Path('/local/sieg/projekte/microminer_evaluation')
sys.path.insert(0, str(root_dir.resolve()))

import helper
from helper.data_operations import read_microminer_csv, merge_results_for_pair_eval
from helper import BAD_PDBIDS


In [2]:
dataset_collection = helper.get_dataset_collection()
supported_datasets = [
    dataset.name
    for dataset in dataset_collection.get_mutation_datasets_with_structure_pairs()
]

In [3]:
query_ids_set = set() 
for dataset_name in supported_datasets:
    dataset = dataset_collection.get_dataset(dataset_name)
    print(f"Evaluating known mutations of {dataset.name}")
    df_ref = dataset.read_single_mutations(pdb_mutant_only=True)
    df_ref = df_ref[~df_ref['wild_pdb'].isin(BAD_PDBIDS)]
    df_ref = df_ref[~df_ref['mut_pdb'].isin(BAD_PDBIDS)]
    query_ids_set.update(df_ref['wild_pdb'].str.lower() + '_' + (df_ref['wild_chain'] if 'wild_chain' in df_ref.columns else 'A'))

# only sample in protherm that is not chain 'A' is 2CI2. Just add manually
query_ids_set.remove('2ci2_A')
query_ids_set.add('2ci2_I')

# in platinum the PDB files are customly prepared. We have to adjust the chain identifier to match the seqres sequences
query_ids_set.remove('3pca_I')
query_ids_set.add('3pca_A')

print(f'collected {len(query_ids_set)} unique query ids')

Evaluating known mutations of protherm
Evaluating known mutations of platinum
Evaluating known mutations of thermomutdb
Evaluating known mutations of shanthirabalan
collected 123 unique query ids


In [4]:
dtypes_dict = {'pdb_wild': str, 'pdb_mutant': str, 'wild_chain': str, 'wild_aa': str, 'seq_num': str, 'mutation_aa': str, 'reason': str}
df_problematic = pd.read_csv(root_dir / 'data' / 'problematic_single_mutations.tsv', sep='\t', header=0, dtype=dtypes_dict)

Download the PDB seqres fasta file

In [5]:
WORK_DIR = Path("/local/sieg/mmseqs_workdir")
QUERY_SET_PATH = WORK_DIR / 'pdb_seqres_query.txt'

In [6]:
%%sh

WORK_DIR="/local/sieg/mmseqs_workdir"
mkdir -p ${WORK_DIR}

# download seqres from PDB
wget https://ftp.wwpdb.org/pub/pdb/derived_data/pdb_seqres.txt.gz -P ${WORK_DIR} -nv
gunzip "${WORK_DIR}/pdb_seqres.txt.gz"

# # download PDB100 from colabfold
# wget https://wwwuser.gwdg.de/~compbiol/colabfold/pdb100_230517.fasta.gz -P ${WORK_DIR} -nv

2023-08-26 10:31:13 URL:https://ftp.wwpdb.org/pub/pdb/derived_data/pdb_seqres.txt.gz [47185569/47185569] -> "/local/sieg/mmseqs_workdir/pdb_seqres.txt.gz" [1]


extract sequences relevant for us

In [7]:
from Bio import SeqIO

def extract_sequences(fasta_filename: str, pdb_ids: set[str], out_fasta_filehandle):
    assert len(pdb_ids) > 0
    id_len = len(next(iter(pdb_ids)))
    for record in SeqIO.parse(fasta_filename, "fasta"):
        pdb_id = record.id[:id_len] #assuming header begins with lowercase PDB id
        if pdb_id in pdb_ids:
            SeqIO.write(record, out_fasta_filehandle, "fasta")
with open(QUERY_SET_PATH, 'w'): pass # clear file content
with open(QUERY_SET_PATH, 'a') as out_fasta_file:
    extract_sequences(WORK_DIR / 'pdb_seqres.txt', {query_id[:4].lower() + query_id[4:] for query_id in query_ids_set}, out_fasta_file)

In [8]:
%%sh

WORK_DIR="/local/sieg/mmseqs_workdir/"

mmseqs createdb "${WORK_DIR}/pdb_seqres_query.txt" "${WORK_DIR}/queryDB"
mmseqs createdb "${WORK_DIR}/pdb_seqres.txt" "${WORK_DIR}/targetDB"

mkdir -p "${WORK_DIR}/results"

mmseqs search "${WORK_DIR}/queryDB" "${WORK_DIR}/targetDB" "${WORK_DIR}/results/resultDB" "${WORK_DIR}/tmp" -a --max-seqs 1000000 #--cov-mode 0 --min-seq-id 0 -s 5.7
 
mmseqs convertalis --format-mode 4 --format-output "query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,qcov,tcov,cigar"\
                   "${WORK_DIR}/queryDB" "${WORK_DIR}/targetDB" "${WORK_DIR}/results/resultDB" "${WORK_DIR}/results/resultDB.tab"


createdb /local/sieg/mmseqs_workdir//pdb_seqres_query.txt /local/sieg/mmseqs_workdir//queryDB 

MMseqs Version:       	407b315e7edcbc9eb73527b904172e603095494e
Database type         	0
Shuffle input database	true
Createdb mode         	0
Write lookup file     	1
Offset of numeric ids 	0
Compressed            	0
Verbosity             	3

Converting sequences
[
Time for merging to queryDB_h: 0h 0m 0s 1ms
Time for merging to queryDB: 0h 0m 0s 1ms
Database type: Aminoacid
Time for processing: 0h 0m 0s 5ms
createdb /local/sieg/mmseqs_workdir//pdb_seqres.txt /local/sieg/mmseqs_workdir//targetDB 

MMseqs Version:       	407b315e7edcbc9eb73527b904172e603095494e
Database type         	0
Shuffle input database	true
Createdb mode         	0
Write lookup file     	1
Offset of numeric ids 	0
Compressed            	0
Verbosity             	3

Converting sequences
[==================================================================================
Time for merging to targetDB_h: 0h 0m 0s 68ms
Time for


Compute score, coverage and sequence identity
Query database size: 123 type: Aminoacid
Target database size: 821414 type: Aminoacid
Calculation of alignments
[=================================================================] 123 2s 863ms
Time for merging to resultDB: 0h 0m 0s 0ms
427118 alignments calculated
146068 sequence pairs passed the thresholds (0.341985 of overall calculated)
1187.544678 hits per query sequence
Time for processing: 0h 0m 3s 352ms
convertalis --format-mode 4 --format-output query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,qcov,tcov,cigar /local/sieg/mmseqs_workdir//queryDB /local/sieg/mmseqs_workdir//targetDB /local/sieg/mmseqs_workdir//results/resultDB /local/sieg/mmseqs_workdir//results/resultDB.tab 

MMseqs Version:        	407b315e7edcbc9eb73527b904172e603095494e
Substitution matrix    	aa:blosum62.out,nucl:nucleotide.out
Alignment format       	4
Format alignment output	query,target,fident,alnlen,mismatch,gapopen,qstart,qend

Create a subset of PDBs relevant for us from the all-vs-all experiment on the PDB 

In [9]:
MMSEQS_RELEVANT_HITS = WORK_DIR / 'results' / 'resultDB.tab'

In [10]:
df_mmseqs = pd.read_csv(MMSEQS_RELEVANT_HITS, sep='\t')
df_mmseqs['target_pdbid'] = df_mmseqs['target'].str[:4]
df_mmseqs

,query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,qcov,tcov,cigar,target_pdbid
0,1amk_A,1amk_A,1.000,251,0,0,1,251,1,251,1.309000e-162,507,1.000,1.000,251M,1amk
1,1amk_A,1if2_A,0.996,251,1,0,1,251,1,251,3.371000e-162,506,1.000,1.000,251M,1if2
2,1amk_A,2vxn_A,0.996,251,1,0,1,251,1,251,3.371000e-162,506,1.000,1.000,251M,2vxn
3,1amk_A,7abx_A,0.996,251,1,0,1,251,1,251,3.371000e-162,506,1.000,1.000,251M,7abx
4,1amk_A,7az3_A,0.996,251,1,0,1,251,1,251,3.371000e-162,506,1.000,1.000,251M,7az3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146063,1nje_A,1b02_A,0.321,314,150,5,3,304,5,267,2.012000e-43,166,0.956,0.943,15M10D25M1I27M1D19M50I17M1D148M,1b02
146064,1nje_A,3v8h_A,0.322,360,155,8,6,316,8,327,1.168000e-39,154,0.984,0.979,83M2D12M40I13M16D5M18D47M2D80M3D2M5D3M3D26M,3v8h
146065,1nje_A,3v8h_B,0.322,360,155,8,6,316,8,327,1.168000e-39,154,0.984,0.979,83M2D12M40I13M16D5M18D47M2D80M3D2M5D3M3D26M,3v8h
146066,1nje_A,3v8h_C,0.322,360,155,8,6,316,8,327,1.168000e-39,154,0.984,0.979,83M2D12M40I13M16D5M18D47M2D80M3D2M5D3M3D26M,3v8h


In [11]:
# stack all individual benchmark data sets to a single dataframe

df_ref_list = []
for dataset_name in supported_datasets:
    dataset = dataset_collection.get_dataset(dataset_name)
            
    print(f"Evaluating known mutations of {dataset.name}")
    df_ref = dataset.read_single_mutations(pdb_mutant_only=True)
    
    df_ref = df_ref[~df_ref['wild_pdb'].isin(BAD_PDBIDS)]
    df_ref = df_ref[~df_ref['mut_pdb'].isin(BAD_PDBIDS)]
    
    df_ref['dataset'] = dataset_name
    
    if dataset_name == "protherm":
        # protherm has no chain identifiers. However, mmseqs needs them. Also its 'A' in all but one case
        df_ref[helper.WILD_CHAIN] = 'A'
        df_ref.loc[df_ref[helper.WILD_COL] == "2CI2", helper.WILD_CHAIN] = 'I'
    
    df_ref = df_ref[[helper.WILD_COL, helper.MUTANT_COL, helper.WILD_AA, helper.MUT_AA, helper.WILD_SEQ_NUM, helper.WILD_CHAIN, 'dataset']]
    df_ref = df_ref.drop_duplicates()
    df_ref_list.append(df_ref)
df_ref = pd.concat(df_ref_list).reset_index(drop=True)
df_ref

Evaluating known mutations of protherm
Evaluating known mutations of platinum
Evaluating known mutations of thermomutdb
Evaluating known mutations of shanthirabalan


,wild_pdb,mut_pdb,wild_aa,mut_aa,wild_seq_num,wild_chain,dataset
0,2RN2,1RBT,K,G,95,A,protherm
1,2RN2,1RBV,K,A,95,A,protherm
2,2RN2,1RBU,K,N,95,A,protherm
3,1IOB,1HIB,T,G,9,A,protherm
4,1BNI,1BRH,L,A,14,A,protherm
...,...,...,...,...,...,...,...
1428,4BFL,3TTT,F,Y,413,A,shanthirabalan
1429,4BFL,4ENP,E,A,530,A,shanthirabalan
1430,4BFL,4ENS,E,Q,530,A,shanthirabalan
1431,4BFL,4ENV,S,I,234,A,shanthirabalan


In [12]:
# add identifiers that can be matched against mmseqs identifiers We use <PDB-id>_<chain-id> for the query and only the pdbid as prefix for the target.
df_ref['mmseqs_wild_ids'] = df_ref['wild_pdb'].str.lower() + '_' + df_ref['wild_chain']
df_ref['mmseqs_mut_id_prefixes'] = df_ref['mut_pdb'].str.lower() 
df_ref

,wild_pdb,mut_pdb,wild_aa,mut_aa,wild_seq_num,wild_chain,dataset,mmseqs_wild_ids,mmseqs_mut_id_prefixes
0,2RN2,1RBT,K,G,95,A,protherm,2rn2_A,1rbt
1,2RN2,1RBV,K,A,95,A,protherm,2rn2_A,1rbv
2,2RN2,1RBU,K,N,95,A,protherm,2rn2_A,1rbu
3,1IOB,1HIB,T,G,9,A,protherm,1iob_A,1hib
4,1BNI,1BRH,L,A,14,A,protherm,1bni_A,1brh
...,...,...,...,...,...,...,...,...,...
1428,4BFL,3TTT,F,Y,413,A,shanthirabalan,4bfl_A,3ttt
1429,4BFL,4ENP,E,A,530,A,shanthirabalan,4bfl_A,4enp
1430,4BFL,4ENS,E,Q,530,A,shanthirabalan,4bfl_A,4ens
1431,4BFL,4ENV,S,I,234,A,shanthirabalan,4bfl_A,4env


In [13]:
df_ref_no_shanthi = df_ref.query('dataset != "shanthirabalan"')
mut_key_wochain = [helper.WILD_COL, helper.MUTANT_COL, helper.WILD_AA, helper.MUT_AA, helper.WILD_SEQ_NUM]
mut_key_wchain = mut_key_wochain + [helper.WILD_CHAIN]

nof_unique_query_chain_target_structure_pairs = df_ref.drop_duplicates(['mmseqs_wild_ids', 'mmseqs_mut_id_prefixes']).shape[0]
nof_unique_mutations_wo_chain = df_ref.drop_duplicates(mut_key_wochain).shape[0]
nof_unique_mutations_w_chain = df_ref.drop_duplicates(mut_key_wchain).shape[0]
print(f"The union of all benchmark data sets contains {nof_unique_query_chain_target_structure_pairs} unique query-chain-target-structure pairs")
print(f"The union of all benchmark data sets contains {nof_unique_mutations_wo_chain} unique mutations (w/o chain)")
print(f"The union of all benchmark data sets contains {nof_unique_mutations_w_chain} unique mutations (w chain)")
print(f"The union of all benchmark data sets contains {df_ref_no_shanthi.drop_duplicates(mut_key_wochain).shape[0]} unique mutations (w/o chain; w/o shanthirabalan)")
print(f"The union of all benchmark data sets contains {df_ref_no_shanthi.drop_duplicates(mut_key_wchain).shape[0]} unique mutations (w chain; w/o shanthirabalan)")

The union of all benchmark data sets contains 1041 unique query-chain-target-structure pairs
The union of all benchmark data sets contains 1136 unique mutations (w/o chain)
The union of all benchmark data sets contains 1137 unique mutations (w chain)
The union of all benchmark data sets contains 590 unique mutations (w/o chain; w/o shanthirabalan)
The union of all benchmark data sets contains 591 unique mutations (w chain; w/o shanthirabalan)


In [14]:
# For mmseqs we can only match <pdb-id>_<chain-id> for the query and pdb-id for the target. Let's remove duplicates to make merging easier.
df_mmseqs_slim = df_mmseqs.sort_values(['alnlen', 'evalue', 'fident'], ascending=[False, True, False]).drop_duplicates(['query', 'target_pdbid'])
df_mmseqs_slim

,query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,qcov,tcov,cigar,target_pdbid
91931,4bfl_A,1iph_A,1.0,753,0,0,1,753,1,753,0.000000,1557,1.000,1.000,753M,1iph
91933,4bfl_A,6by0_A,1.0,753,0,0,1,753,1,753,0.000000,1557,1.000,1.000,753M,6by0
91939,4bfl_A,1gge_A,1.0,753,0,0,1,753,1,753,0.000000,1557,1.000,1.000,753M,1gge
91943,4bfl_A,3vu3_A,1.0,753,0,0,1,753,1,753,0.000000,1557,1.000,1.000,753M,3vu3
91944,4bfl_A,4bfl_A,1.0,753,0,0,1,753,1,753,0.000000,1557,1.000,1.000,753M,4bfl
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124113,2jbz_A,5umw_E,1.0,18,0,0,5,22,1,18,0.000968,42,0.126,0.119,18M,5umw
124235,2jbz_A,1t3d_A,1.0,18,0,0,5,22,1,18,0.000968,42,0.126,0.062,18M,1t3d
124251,2jbz_A,4txd_A,1.0,18,0,0,5,22,1,18,0.000968,42,0.126,0.046,18M,4txd
124272,2jbz_A,7blf_A,1.0,18,0,0,5,22,1,18,0.000968,42,0.126,0.040,18M,7blf


In [15]:
df_ref_w_mmseqs = df_ref.merge(df_mmseqs_slim, left_on=['mmseqs_wild_ids', 'mmseqs_mut_id_prefixes'], right_on=['query', 'target_pdbid'], how='left')
assert df_ref_w_mmseqs.shape[0] == df_ref.shape[0]
df_ref_w_mmseqs

,wild_pdb,mut_pdb,wild_aa,mut_aa,wild_seq_num,wild_chain,dataset,mmseqs_wild_ids,mmseqs_mut_id_prefixes,query,...,qstart,qend,tstart,tend,evalue,bits,qcov,tcov,cigar,target_pdbid
0,2RN2,1RBT,K,G,95,A,protherm,2rn2_A,1rbt,2rn2_A,...,1.0,155.0,1.0,155.0,1.366000e-104,334.0,1.0,1.0,155M,1rbt
1,2RN2,1RBV,K,A,95,A,protherm,2rn2_A,1rbv,2rn2_A,...,1.0,155.0,1.0,155.0,9.961000e-105,334.0,1.0,1.0,155M,1rbv
2,2RN2,1RBU,K,N,95,A,protherm,2rn2_A,1rbu,2rn2_A,...,1.0,155.0,1.0,155.0,7.265000e-105,335.0,1.0,1.0,155M,1rbu
3,1IOB,1HIB,T,G,9,A,protherm,1iob_A,1hib,1iob_A,...,1.0,153.0,1.0,153.0,1.384000e-95,308.0,1.0,1.0,153M,1hib
4,1BNI,1BRH,L,A,14,A,protherm,1bni_A,1brh,1bni_A,...,1.0,110.0,1.0,110.0,1.979000e-70,233.0,1.0,1.0,110M,1brh
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,4BFL,3TTT,F,Y,413,A,shanthirabalan,4bfl_A,3ttt,4bfl_A,...,1.0,753.0,1.0,753.0,0.000000e+00,1556.0,1.0,1.0,753M,3ttt
1429,4BFL,4ENP,E,A,530,A,shanthirabalan,4bfl_A,4enp,4bfl_A,...,1.0,753.0,1.0,753.0,0.000000e+00,1555.0,1.0,1.0,753M,4enp
1430,4BFL,4ENS,E,Q,530,A,shanthirabalan,4bfl_A,4ens,4bfl_A,...,1.0,753.0,1.0,753.0,0.000000e+00,1556.0,1.0,1.0,753M,4ens
1431,4BFL,4ENV,S,I,234,A,shanthirabalan,4bfl_A,4env,4bfl_A,...,1.0,753.0,1.0,753.0,0.000000e+00,1555.0,1.0,1.0,753M,4env


In [16]:
# show what mmseqs did not find
df_mmseqs_not_found = df_ref_w_mmseqs.query('query != query').drop_duplicates(['wild_pdb', 'mut_pdb', 'wild_aa', 'mut_aa', 'wild_seq_num', 'wild_chain'])
df_mmseqs_not_found

,wild_pdb,mut_pdb,wild_aa,mut_aa,wild_seq_num,wild_chain,dataset,mmseqs_wild_ids,mmseqs_mut_id_prefixes,query,...,qstart,qend,tstart,tend,evalue,bits,qcov,tcov,cigar,target_pdbid
14,1BNI,1BGD,I,V,96,A,protherm,1bni_A,1bgd,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,2RN2,1RBN,K,N,95,A,protherm,2rn2_A,1rbn,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262,1LZ1,1YAG,I,V,89,A,protherm,1lz1_A,1yag,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
278,1LZ1,1GA2,V,I,2,A,protherm,1lz1_A,1ga2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
279,1LZ1,1GA0,V,L,2,A,protherm,1lz1_A,1ga0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
295,1LZ1,1GA0,V,L,74,A,protherm,1lz1_A,1ga0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
300,1LZ1,1GA0,V,L,110,A,protherm,1lz1_A,1ga0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
371,3PCA,1YKL,Y,C,408,I,platinum,3pca_I,1ykl,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
372,3PCA,1YKP,Y,H,408,I,platinum,3pca_I,1ykp,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
816,1R2Y,3GQ9,R,E,244,A,thermomutdb,1r2y_A,3gq9,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# compare benchmark pairs mmseqs not find with the set of problematic dataset mutation identified previousely
df_mmseqs_not_found.merge(df_problematic, left_on=[helper.WILD_COL, helper.MUTANT_COL, helper.WILD_AA, helper.MUT_AA, helper.WILD_SEQ_NUM],
                         right_on= ['pdb_wild', 'pdb_mutant', 'wild_aa', 'mutation_aa', 'seq_num'], how='left', indicator=True)

,wild_pdb,mut_pdb,wild_aa,mut_aa,wild_seq_num,wild_chain_x,dataset,mmseqs_wild_ids,mmseqs_mut_id_prefixes,query,...,tcov,cigar,target_pdbid,pdb_wild,pdb_mutant,wild_chain_y,seq_num,mutation_aa,reason,_merge
0,1BNI,1BGD,I,V,96,A,protherm,1bni_A,1bgd,NaN,...,NaN,NaN,NaN,1BNI,1BGD,A,96,V,different proteins,both
1,2RN2,1RBN,K,N,95,A,protherm,2rn2_A,1rbn,NaN,...,NaN,NaN,NaN,2RN2,1RBN,A,95,N,different proteins,both
2,1LZ1,1YAG,I,V,89,A,protherm,1lz1_A,1yag,NaN,...,NaN,NaN,NaN,1LZ1,1YAG,A,89,V,different proteins,both
3,1LZ1,1GA2,V,I,2,A,protherm,1lz1_A,1ga2,NaN,...,NaN,NaN,NaN,1LZ1,1GA2,A,2,I,different proteins,both
4,1LZ1,1GA0,V,L,2,A,protherm,1lz1_A,1ga0,NaN,...,NaN,NaN,NaN,1LZ1,1GA0,A,2,L,different proteins,both
5,1LZ1,1GA0,V,L,74,A,protherm,1lz1_A,1ga0,NaN,...,NaN,NaN,NaN,1LZ1,1GA0,A,74,L,different proteins,both
6,1LZ1,1GA0,V,L,110,A,protherm,1lz1_A,1ga0,NaN,...,NaN,NaN,NaN,1LZ1,1GA0,A,110,L,different proteins,both
7,3PCA,1YKL,Y,C,408,I,platinum,3pca_I,1ykl,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
8,3PCA,1YKP,Y,H,408,I,platinum,3pca_I,1ykp,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
9,1R2Y,3GQ9,R,E,244,A,thermomutdb,1r2y_A,3gq9,NaN,...,NaN,NaN,NaN,1R2Y,3GQ9,A,244,E,different proteins,both


As we can see all pairs not found by MMseqs are in the problematic set. Only 3PCA-1YKL and 3PCA-1YKP is not found because of the custom chain identifiers in platinum. (see the _merge column)

However, when we search for 3pca_A in the mmseqs results we find the expected hits.

In [18]:
df_mmseqs.query('(query == "3pca_A" and target_pdbid == "1ykl") or (query == "3pca_A" and target_pdbid == "1ykp")')

,query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,qcov,tcov,cigar,target_pdbid
54455,3pca_A,1ykp_A,1.0,200,0,0,1,200,1,200,9.231000e-135,424,1.000,1.000,200M,1ykp
54461,3pca_A,1ykp_C,1.0,200,0,0,1,200,1,200,9.231000e-135,424,1.000,1.000,200M,1ykp
54467,3pca_A,1ykp_E,1.0,200,0,0,1,200,1,200,9.231000e-135,424,1.000,1.000,200M,1ykp
54472,3pca_A,1ykp_G,1.0,200,0,0,1,200,1,200,9.231000e-135,424,1.000,1.000,200M,1ykp
54476,3pca_A,1ykp_I,1.0,200,0,0,1,200,1,200,9.231000e-135,424,1.000,1.000,200M,1ykp
54481,3pca_A,1ykp_K,1.0,200,0,0,1,200,1,200,9.231000e-135,424,1.000,1.000,200M,1ykp
54539,3pca_A,1ykl_A,1.0,200,0,0,1,200,1,200,9.231000e-135,424,1.000,1.000,200M,1ykl
54543,3pca_A,1ykl_C,1.0,200,0,0,1,200,1,200,9.231000e-135,424,1.000,1.000,200M,1ykl
54549,3pca_A,1ykl_E,1.0,200,0,0,1,200,1,200,9.231000e-135,424,1.000,1.000,200M,1ykl
54555,3pca_A,1ykl_G,1.0,200,0,0,1,200,1,200,9.231000e-135,424,1.000,1.000,200M,1ykl


In [19]:
nof_query_chain_target_structures_with_successful_mmseqs_hit = df_ref_w_mmseqs.query('query == query').drop_duplicates(['mmseqs_wild_ids', 'mmseqs_mut_id_prefixes']).shape[0]
nof_unique_query_chain_target_structure_pairs_adjusted = nof_unique_query_chain_target_structure_pairs - 2 # adjust for 3pca_I in platinum
print(f'MMseqs successfully retrieved {100*(nof_query_chain_target_structures_with_successful_mmseqs_hit / nof_unique_query_chain_target_structure_pairs_adjusted):.2f}% query_chain_target_structures pairs')
print(f'MMseqs missed {nof_unique_query_chain_target_structure_pairs_adjusted - nof_query_chain_target_structures_with_successful_mmseqs_hit} query_chain_target_structures pairs')

MMseqs successfully retrieved 99.42% query_chain_target_structures pairs
MMseqs missed 6 query_chain_target_structures pairs


## Compare MMseqs results to 'problematic_single_mutations' 

MMseqs hits include hits that we don't want to find because of local differences in the 3D env and generally errors in the annotations of the datasets 

In [20]:
nof_query_chain_target_structures_with_successful_mmseqs_hit

1033

In [21]:
df_ref_w_mmseqs_prob = df_ref_w_mmseqs.merge(df_problematic, left_on=[helper.WILD_COL, helper.MUTANT_COL, helper.WILD_AA, helper.MUT_AA, helper.WILD_SEQ_NUM, helper.WILD_CHAIN],
                         right_on= ['pdb_wild', 'pdb_mutant', 'wild_aa', 'mutation_aa', 'seq_num', 'wild_chain'], how='left', indicator=True).query('_merge == "both" and query== query')
nof_problematic_chain_pairs_reported_by_mmseqs = df_ref_w_mmseqs_prob.drop_duplicates(["mmseqs_wild_ids", "mmseqs_mut_id_prefixes"]).shape[0]
print(f'MMseqs reported {100*(nof_problematic_chain_pairs_reported_by_mmseqs / nof_query_chain_target_structures_with_successful_mmseqs_hit):.2f}% \'problematic mutation\' query_chain_target_structures pairs')
print('nof_problematic_chain_pairs_reported_by_mmseqs=', nof_problematic_chain_pairs_reported_by_mmseqs)


MMseqs reported 9.10% 'problematic mutation' query_chain_target_structures pairs
nof_problematic_chain_pairs_reported_by_mmseqs= 94


In [22]:
nof_problematic_mutations_reported_by_mmseqs = df_ref_w_mmseqs_prob.drop_duplicates([helper.WILD_COL, helper.MUTANT_COL, helper.WILD_AA, helper.MUT_AA, helper.WILD_SEQ_NUM, helper.WILD_CHAIN]).shape[0]
print('nof_problematic_mutations_reported_by_mmseqs=', nof_problematic_mutations_reported_by_mmseqs)
print(f'these are {100*(nof_problematic_mutations_reported_by_mmseqs / nof_unique_mutations_wo_chain):.2f}% of all mutations')

nof_problematic_mutations_reported_by_mmseqs= 178
these are 15.67% of all mutations


In [23]:
df_ref_w_mmseqs_prob.query('reason == "different proteins"')

,wild_pdb,mut_pdb,wild_aa,mut_aa,wild_seq_num,wild_chain,dataset,mmseqs_wild_ids,mmseqs_mut_id_prefixes,query,...,qcov,tcov,cigar,target_pdbid,pdb_wild,pdb_mutant,seq_num,mutation_aa,reason,_merge
833,1PGA,6HKA,A,L,34,A,thermomutdb,1pga_A,6hka,1pga_A,...,1.0,0.56,56M,6hka,1PGA,6HKA,34,L,different proteins,both


## MMseqs reports "different" protein in one case

TMAlign and MicroMiner don't report the mutation "1PGA	6HKA	A	A	34	L" from thermomutdb. But MMseqs reports a hit for 1pga_A with 6hka_A.


In [24]:
# look at the seqres fasta sequences of both files
!grep -A1 1pga /local/sieg/mmseqs_workdir/pdb_seqres.txt
!grep -A1 6hka /local/sieg/mmseqs_workdir/pdb_seqres.txt

>1pga_A mol:protein length:56  PROTEIN G
MTYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE
>6hka_A mol:protein length:100  Immunoglobulin G-binding protein G,Serine-protein kinase ATM
MQYKLALNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTELVPRGSDDDDKTVLSVGGQVNLLIQQAIDPKNLSRLFPGWKAWV


In [25]:
from Bio.PDB import PDBList, PDBParser, Polypeptide
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

def download_and_extract(pdb_id, chain_id):
    pdbl = PDBList()
    pdb_file = pdbl.retrieve_pdb_file(pdb_id, pdir=str(WORK_DIR.resolve()), file_format="pdb")

    parser = PDBParser(QUIET=True)
    structure = parser.get_structure(pdb_id, pdb_file)

    atom_sequence = ""
    models = list(structure.get_models())
    model = models[0]
    for chain in model:
        if chain.id != chain_id:
            continue
        for residue in chain:
            for atom in residue:
                if atom.get_id() == 'CA':
                    atom_sequence += Polypeptide.three_to_one(residue.get_resname())

    return atom_sequence

def write_to_fasta(sequence, entry_id, out_dir):
    fasta_record = SeqRecord(Seq(sequence), id=entry_id, description=f"{entry_id} atom sequence")
    fasta_file = f"{entry_id}.fasta"
    with open(out_dir / fasta_file, "w") as output_handle:
        output_handle.write(fasta_record.format("fasta"))  

atomseq_1pga = download_and_extract('1pga', 'A')
atomseq_6hka = download_and_extract('6hka', 'A')

write_to_fasta(atomseq_1pga, '1pga_A', WORK_DIR)
write_to_fasta(atomseq_6hka, '6hka_A', WORK_DIR)
 
print('1pga', atomseq_1pga)
print('6hka', atomseq_6hka)

1pga MTYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE
6hka TVLSVGGQVNLLIQQAIDPKNLSRLFPGWKAWV


As we can see above while the atom sequence of 1pga is identical to the seqres sequence, the atom sequence extracted from 6hka is very different and contains only the second half of the sequence  

In [26]:
%%sh

# lets run MMseqs on the atom sequences

WORK_DIR="/local/sieg/mmseqs_workdir/"

mmseqs createdb "${WORK_DIR}/1pga_A.fasta" "${WORK_DIR}/1pga_A_DB"
mmseqs createdb "${WORK_DIR}/6hka_A.fasta" "${WORK_DIR}/6hka_A_DB"

mkdir -p "${WORK_DIR}/results"

mmseqs search "${WORK_DIR}/1pga_A_DB" "${WORK_DIR}/6hka_A_DB" "${WORK_DIR}/results/1pga_A_6hka_A_resultDB" "${WORK_DIR}/tmp" -a --max-seqs 1000000 #--cov-mode 0 --min-seq-id 0 -s 5.7
 
mmseqs convertalis --format-mode 4 --format-output "query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,qcov,tcov,cigar"\
                   "${WORK_DIR}/1pga_A_DB" "${WORK_DIR}/6hka_A_DB" "${WORK_DIR}/results/1pga_A_6hka_A_resultDB" "${WORK_DIR}/results/1pga_A_6hka_A_resultDB.tab"


createdb /local/sieg/mmseqs_workdir//1pga_A.fasta /local/sieg/mmseqs_workdir//1pga_A_DB 

MMseqs Version:       	407b315e7edcbc9eb73527b904172e603095494e
Database type         	0
Shuffle input database	true
Createdb mode         	0
Write lookup file     	1
Offset of numeric ids 	0
Compressed            	0
Verbosity             	3

Converting sequences
[
Time for merging to 1pga_A_DB_h: 0h 0m 0s 1ms
Time for merging to 1pga_A_DB: 0h 0m 0s 0ms
Database type: Aminoacid
Time for processing: 0h 0m 0s 3ms
createdb /local/sieg/mmseqs_workdir//6hka_A.fasta /local/sieg/mmseqs_workdir//6hka_A_DB 

MMseqs Version:       	407b315e7edcbc9eb73527b904172e603095494e
Database type         	0
Shuffle input database	true
Createdb mode         	0
Write lookup file     	1
Offset of numeric ids 	0
Compressed            	0
Verbosity             	3

Converting sequences
[
Time for merging to 6hka_A_DB_h: 0h 0m 0s 0ms
Time for merging to 6hka_A_DB: 0h 0m 0s 0ms
Database type: Aminoacid
Time for processing: 0h 


Compute score, coverage and sequence identity
Query database size: 1 type: Aminoacid
Target database size: 1 type: Aminoacid
Calculation of alignments
[=================================================================] 1 0s 0ms
Time for merging to 1pga_A_6hka_A_resultDB: 0h 0m 0s 0ms
0 alignments calculated
0 sequence pairs passed the thresholds
0.000000 hits per query sequence
Time for processing: 0h 0m 0s 0ms
convertalis --format-mode 4 --format-output query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,qcov,tcov,cigar /local/sieg/mmseqs_workdir//1pga_A_DB /local/sieg/mmseqs_workdir//6hka_A_DB /local/sieg/mmseqs_workdir//results/1pga_A_6hka_A_resultDB /local/sieg/mmseqs_workdir//results/1pga_A_6hka_A_resultDB.tab 

MMseqs Version:        	407b315e7edcbc9eb73527b904172e603095494e
Substitution matrix    	aa:blosum62.out,nucl:nucleotide.out
Alignment format       	4
Format alignment output	query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,e

In [27]:
!cat /local/sieg/mmseqs_workdir/results/1pga_A_6hka_A_resultDB.tab

query	target	fident	alnlen	mismatch	gapopen	qstart	qend	tstart	tend	evalue	bits	qcov	tcov	cigar


As we can observe, when we use the atom sequence with MMseqs (the same sequence that is used by MicroMiner and TMalign) no hits are reported for the chain pari 1pga_A 6hka_A